In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import os
import time
import re
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from pearlsim.ml_utilities import *
import pickle

# Extract data from Serpent detector files
We can use the read_det_file function from ml_utilities to parse all of the detector files we have in the training data directory to create one pair of unified features/target dataframes.

Sometimes the file(s) provided will have really high uncertainty. This will absolutely limit your model's accuracy, so don't be disheartened until I get you more accurate data, which can take a while to run.

In [2]:
training_steps = [1, 22]
all_features = pd.DataFrame([])
all_targets = pd.DataFrame([])
for i in training_steps:
    # Load the Serpent Detector file, a matlab format file that has pebble surface flux
    # and overall core flux. 
    det_name = f"gFHR_core_{i}.serpent_det0.m"
    
    # Load the auxiliary feature file, which are additionally features I included from
    # the model that tell you more about the respective pebble for each surface
    aux_name = f"current_auxiliary_features{i}.csv"
    features, targets, pebble_ids, avg_uncertainty = read_det_file("training_data/"+det_name)
    aux_features = pd.read_csv("training_data/"+aux_name, index_col=0)
    
    # Read the pebble power file and grab the corresponding power values
    pow_name = f"pebble_positions_{i}.csv_pow0.m"
    pow_data = pd.read_csv("training_data/"+pow_name, delimiter='\s+', 
                           names=['x','y','z','rad','universe','power','unc'])
    targets['power'] = pow_data['power'].iloc[pebble_ids].reset_index().drop(columns='index')
    features = aux_features.join(features)
    
    # Convert coordinates to 2D cylindrical, since the flux map is symmetrical
    features['radius'] = round(np.sqrt(features['x']**2+features['y']**2),1)
    features['height'] = round(features['z'],1)
    features = features.drop(columns=['x','y','z'])
    
    print(f"File {det_name} has an average {round(avg_uncertainty*100,2)}% uncertainty.")
    all_features = pd.concat([all_features, features])
    all_targets = pd.concat([all_targets, targets])

# For clarity, re-order the radius and height columns to be first and save
headers = all_features.columns.to_list()
headers.remove("radius")
headers.remove("height")
all_features = all_features[["radius"]+["height"]+headers]

all_features.to_csv("training_data/current_data.csv")
all_targets.to_csv("training_data/current_target.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'training_data/gFHR_core_1.serpent_det0.m'

Our features include the radius and height of the pebble-based detector. It also includes every bin of the core flux map. The gFHR model is divided into 12 energy groups and has 4 radial divisions, each divided into 10 separate axial zones. Normally you would need to volume-weight these flux values, and deal with the somewhat complicated indexing scheme to sort out which is which. We don't need to bother here, since they're all going to be standardized anyways and the model will make its own inferences about the spatial distribution.

In [ ]:
all_features.iloc[-10:-1]

Meanwhile, the target variables represent the neutron current going into the pebbles above, divided into the same 12 energy bins. There's a tradeoff between how fine our energy grid is, and how long it takes for the Serpent model to run. Every time you add a new bin, you need to simulate more particles to get sufficient statistics. So, our goal is to maximize accuracy using as few bins as possible.

In [ ]:
all_targets.iloc[0:10]

In [ ]:
i_to_plot = 2
energies = all_targets.drop(columns="power").columns.to_list()
sample_current = all_targets.drop(columns="power").iloc[i_to_plot].to_list()
plt.plot(energies, sample_current)
plt.yscale("log")
plt.xscale("log")
plt.xlabel("Energy MeV")
plt.ylabel("Pebble Surface Current (n/cm^2-s)")
plt.show()
print(f"Pebble power: {all_targets.iloc[i_to_plot]['power']}")

# Data Standardization
Simple standardization is performed here along each column. I tried log-standardization, but it didn't seem to help with the current values.

In [ ]:
train_split = 0.8
np.random.seed(42)

def standardize(raw_data, mean=None, std=None, axis=0):
    if mean is None:
        mean = np.mean(raw_data, axis = axis)
    if std is None:
        std = np.std(raw_data, axis = axis)
        std[ std==0 ] = 0.1
    result = (raw_data - mean) / std
    return result, mean, std

def unstandardize(standardized_data, mean, std):
    raw_data = (standardized_data*std)+mean
    return raw_data

#log_features = all_features#.apply(lambda x: np.log10(x + 1))
#log_targets = all_targets
#log_targets.iloc[:,2:] = all_targets.iloc[:,2:]#.apply(lambda x: np.log10(x + 1))

num_data = len(all_features)
training_size = int(num_data*train_split)
testing_size = num_data - training_size
data_indices = np.arange(num_data)
training_indices = np.random.choice(num_data, training_size, replace=False)
testing_indices = data_indices[np.in1d(data_indices, training_indices, invert=True)]

training_data, data_mean, data_std = standardize(all_features.iloc[training_indices])
training_target, target_mean, target_std = standardize(all_targets.iloc[training_indices])
testing_data, _, _  = standardize(all_features.iloc[testing_indices], mean=data_mean, std=data_std)
testing_target, _, _  = standardize(all_targets.iloc[testing_indices], mean=target_mean, std=target_std)

print(np.shape(training_data))
print(np.shape(training_target))
print(np.shape(testing_data))
print(np.shape(testing_target))

# Model Training
I threw together a quick RFR model and got some results. You're free to change to any other type of model, as long as its something I can save and load into other modules. Things to try:
- Properly using cross validation
- Tuning the hyper parameters
- Trying a different model, probably a neural net

In [ ]:
best_params = {'max_depth': 10, 
               'n_estimators': 1000, 
               'n_jobs': 14,} # Set to your number of cores
rfr_model = RandomForestRegressor(random_state=0)
rfr_model.set_params(**best_params)
rfr_model.fit(training_data, training_target)
rfr_model_test_score = rfr_model.score(testing_data, testing_target)
print(f"RFR score: {rfr_model_test_score}")

Next we save the models and standardization parameters so the model can be used in the simulation.

In [ ]:
model_data = pickle.dumps(rfr_model)
with open("ml_models/current_rfr.pkl", 'wb') as f:
    f.write(model_data)
data_mean.to_csv("ml_models/current_rfr_data_mean.csv", header=True)
data_std.to_csv("ml_models/current_rfr_data_std.csv", header=True)
target_mean.to_csv("ml_models/current_rfr_target_mean.csv", header=True)
target_std.to_csv("ml_models/current_rfr_target_std.csv", header=True)